# 機械学習ハンズオン（MNIST画像分類編）

【課題】 MNIST（手書き数字データセット）の画像を0～9の数字に分類しましょう。

ここでは、次の3つの方法を試します。
 1. 1層パーセプトロン
 1. 多層（…と言っても2層）パーセプトロン
 1. 2層の畳み込みニューラルネットワーク(CNN)

## 事前準備（最初に一度だけ実行してください）

### ライブラリのロード
Binderで実行する場合は、1行目のコメントアウトを外して実行してください。

In [0]:
# !pip install tensorflow matplotlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

### データセット(MNIST)の読み込み
データをダウンロードします。
データは訓練データとテストデータに分かれて取得されます。

In [0]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

### データの正規化
MNISTのデータは各ピクセルが0-255の値を持ったモノクロデータですが、機械学習の効率を上げるため、これを0.0-1.0の範囲になるように数値を調整します。

（注意）ここを複数回実行すると、その分だけ値が小さくなってしまいます。

In [0]:
x_train, x_test = x_train / 255.0, x_test / 255.0

調整後の画像データはこんな感じになります。

In [0]:
print(x_train[0])

## 1層パーセプトロン
 * モデル
  1. 28*28の配列をフラット化する。
  2. 入力が784ノード、出力が10ノードの全結合層を作る。この層の出力をsoftmaxで確率に変換する。
 * 最適化
  * Adamアルゴリズム
 * 損失
  * 交差エントロピー

### モデルの作成
Kerasの学習モデルを作ります。

In [0]:
model_slp = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model_slp.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [0]:
model_slp.summary()

### モデルの学習
モデルに訓練データを渡して学習します。

In [0]:
model_slp.fit(x_train, y_train, epochs=5)

### モデルの評価
テストデータ（学習モデルにとっては未知のデータ）を使って、学習モデルを評価します。

In [0]:
model_slp.evaluate(x_test, y_test)

## 多層パーセプトロン
 * モデル
  1. 28*28の配列をフラット化する。
  2. 入力が784ノード、出力が512ノードの全結合層を作る。この層の出力をReLUでアクティベーションする（要するに負の値を0に変換している）。
  3. 過学習を防ぐためにドロップアウト層を作る。（全ノードの20%だけ残して後を無効にする）
  4. 入力が512ノード、出力が10ノードの全結合層を作る。この層の出力をsoftmaxで確率に変換する。
 * 最適化
  * 1層と同じ
 * 損失
  * 1層と同じ

### モデルの作成

In [0]:
model_mlp = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model_mlp.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [0]:
model_mlp.summary()

### モデルの学習

In [0]:
model_mlp.fit(x_train, y_train, epochs=5)

### モデルの評価

In [0]:
model_mlp.evaluate(x_test, y_test)

## 畳み込みニューラルネットワーク

 * モデル
   1. 5x5のフィルターを32個作って畳み込みを行う。
   1. 2x2で最大プーリングする。
   1. さらに5x5のフィルターを64個作って畳み込みを行う。
   1. さらに2x2で最大プーリングする。
   1. いったんフラット化する。
   1. 出力1024ノードの全結合層を作る。
   1. 出力10ノードの全結合層を作る。この出力をsoftmaxで確率に変換する。

In [0]:
model_cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=[5, 5], padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=[2, 2]),
    tf.keras.layers.Conv2D(filters=64, kernel_size=[5, 5], padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=[2, 2]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model_cnn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

### モデルの学習

In [0]:
x_train_cnn = x_train.reshape(x_train.shape[0], 28, 28, 1)
model_cnn.fit(x_train_cnn, y_train, epochs=5)

In [0]:
# もしここでエラーが出てしまったら、このセルを再実行するか、このセルを無視してください。
model_cnn.summary()

### モデルの評価

In [0]:
x_test_cnn = x_test.reshape(x_test.shape[0], 28, 28, 1)
model_cnn.evaluate(x_test_cnn, y_test)